<a href="https://colab.research.google.com/github/ZeyadOsama/tensorflow-handy-snippets/blob/master/visualizations/Visualizing_Intermediate_Representations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualizing Intermediate Representations

### Description


To get a feel for what kind of features ConvNet has learned, visualize how an input gets transformed as it goes through the convnet.

### Code Snippet

In [0]:
import tensorflow.keras.models

def visualize_layers(model: tensorflow.keras.models, img_path: str, target_size:()):
    """
    To get a feel for what kind of features ConvNet has learned,
    visualize how an input gets transformed as it goes through the convnet.

    :param model: compiled model
    :param img_path: image path to visulize
    :param target_size: image size
    """
    import numpy as np
    from tensorflow.keras.preprocessing.image import img_to_array, load_img

    # Define a new Model that will take an image as input, and will output
    # intermediate representations for all layers in the previous model after
    # the first.
    successive_outputs = [layer.output for layer in model.layers[1:]]
    visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

    # this is a PIL image
    img = load_img(img_path, target_size=target_size) 

    # Numpy array with shape ((target_size), 3)
    x = img_to_array(img)

    # Numpy array with shape (1, (target_size), 3)
    x = x.reshape((1,) + x.shape)

    # Rescale by 1/255
    x /= 255

    # Run provided image through the network, thus obtaining all
    # intermediate representations for this image.
    successive_feature_maps = visualization_model.predict(x)

    # These are the names of the layers, so can have them as part of the plot
    layer_names = [layer.name for layer in model.layers]

    # Display representations
    for layer_name, feature_map in zip(layer_names, successive_feature_maps):
        if len(feature_map.shape) == 4:
            # Just do this for the conv / maxpool layers
            # not the fully-connected layers
            n_features = feature_map.shape[-1]  # number of features in feature map
            
            # The feature map has shape (1, size, size, n_features)
            size = feature_map.shape[1]
            
            # Tile images in this matrix
            display_grid = np.zeros((size, size * n_features))
            
            for i in range(n_features):
                # Postprocess the feature to make it visually palatable
                x = feature_map[0, :, :, i]
                x -= x.mean()
                x /= x.std()
                x *= 64
                x += 128
                x = np.clip(x, 0, 255).astype('uint8')
                
                # Tile each filter into this big horizontal grid
                display_grid[:, i * size : (i + 1) * size] = x
            
            scale = 20. / n_features
            
            # Display the grid
            plt.figure(figsize=(scale * n_features, scale))
            plt.title(layer_name)
            plt.grid(False)
            plt.imshow(display_grid, aspect='auto', cmap='viridis')
    return